<a href="https://colab.research.google.com/github/jaskiratkalra26/Generative-AI/blob/main/finetuning_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from datasets import load_dataset
data = load_dataset('mlabonne/guanaco-llama2-1k',split='train')

In [3]:
data[0]

{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}

In [4]:
!pip install --q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 -U bitsandbytes

In [5]:
!pip install -U bitsandbytes

  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Successfully uninstalled bitsandbytes-0.40.2


In [6]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype= torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained('NousResearch/Llama-2-7b-chat-hf',
                                             quantization_config = bnb_config,
                                             device_map='auto')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf',trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [8]:
from peft import prepare_model_for_kbit_training,get_peft_model,LoraConfig
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r= 16,
    lora_alpha = 32,
    target_modules = ['q_proj','v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

model = get_peft_model(model,lora_config)

In [10]:
from transformers import TrainingArguments
training_arguments = TrainingArguments(
    output_dir = './output',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps= 4,# for simulating larger batch size
    optim = 'paged_adamw_8bit',
    learning_rate = 2e-4,
    lr_scheduler_type = 'cosine',
    save_strategy = 'epoch',
    logging_steps = 10,
    num_train_epochs= 1,
    warmup_ratio = 0.05,
    fp16 = True

)

In [12]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model = model,
    train_dataset = data,
    peft_config = lora_config,
    dataset_text_field = 'text',
    max_seq_length = 1024,
    tokenizer = tokenizer,
    args = training_arguments,
    packing = True
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:427: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:1301: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  new_forward = torch.cuda.amp.autocast(dtype=torch.float16)(model_forward_func)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jaskirats2004 (jaskirats2004-chandigarh-group-of-colleges) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.758500
20,1.447300


TrainOutput(global_step=28, training_loss=1.5610050133296423, metrics={'train_runtime': 1728.1347, 'train_samples_per_second': 0.579, 'train_steps_per_second': 0.036, 'total_flos': 9380252161671168.0, 'train_loss': 1.5610050133296423, 'epoch': 0.45})

In [14]:
trainer.model.save_pretrained('new_model')

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Determine the device for LoRA layers
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create a manual device map to offload most layers to CPU
device_map = {"": "cpu"} # Offload all layers to CPU by default

# Load the base model with the manual device map
base_model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map, # Pass the manual device map
)

# Load the LoRA adapters with offload_dir
model = PeftModel.from_pretrained(base_model, 'new_model', offload_dir='./offload') # Add offload_dir here

# Move the LoRA model to the determined device (GPU if available)
model.to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf', trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# Create the pipeline
from transformers import pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=200)

# Run the pipeline
res = pipe('write an amendment to articles of incorporation adding new member to an LLC ')
print(res[0]['generated_text'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

write an amendment to articles of incorporation adding new member to an LLC 

I am the founder of an LLC and I want to add a new member to the company. I am not sure how to go about this, as I am not familiar with the legal requirements for adding a new member to an LLC. Can you please provide me with some guidance on how to proceed?

To add a new member to an LLC, you will need to follow the procedures outlined in your state's LLC laws. These procedures typically involve filing an amendment to the LLC's articles of incorporation with the state.

Here are the general steps you will need to follow to add a new member to an LLC:

1. Review your LLC's articles of incorporation: Before you can add a new member to your LLC, you will need to review your LLC's articles of incorporation to


In [4]:
res = pipe("Write an article about treasury management such that its understandable for a child of 10 why it is important")
print(res[0]['generated_text'])


Write an article about treasury management such that its understandable for a child of 10 why it is important to manage money well.
Treasury management is the process of managing money in a business or organization. It involves making sure that the money is used wisely and that it is safe.
Imagine you have a piggy bank where you save your money. You want to make sure that your money is safe and that you can use it when you need it. That's what treasury management is like for a business.
The people who manage the money in a business are called treasurers. They make sure that the money is used wisely and that it is safe. They also make sure that the business has enough money to pay its bills and to buy things it needs.
Just like you keep track of how much money is in your piggy bank, treasurers keep track of how much money the business
